In [2]:
#pip install python-magic-bin
import os
import filetype
import magic
import datetime
import csv
from PIL import Image
from PIL.ExifTags import TAGS
import ffmpeg
import sys
import filetype
from pprint import pprint


def isCsv(dir_path, file):
    afile = os.path.join(dir_path, file)
    with open(afile, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        try:
            for row in spamreader:
                return True
        except Exception as e:
            return False

def check_file_type(dir_path, file):
    afile = os.path.join(dir_path, file)    
    try:
        f = magic.Magic(mime=True)
        file_type = f.from_file(afile)        
        f0 = file_type.split('/')[0]
        if (f0 == 'image') | (f0 == 'video') | (f0 == 'audio'):
            return f0
        elif f0 == 'text':
            if file_type.split('/')[1] == 'plain':
                if isCsv(dir_path, file):
                    return 'csv'
                else:
                    return 'txt'
#             elif file_type.split('/')[1] == 'html':
#                 return 'html'
            else:
                return 'nothing'
        else:
            file_type = filetype.guess(afile).mime            
            f0 = file_type.split('/')[0]
            if (f0 == 'image') | (f0 == 'video') | (f0 == 'audio'):
                return f0
            else:
                return "nothing";        

    except Exception as e:
        pass #print('error') #print(e)

def image_process(dir_path, file):
    afile = os.path.join(dir_path, file)
    image = Image.open(afile)
    info_dict = {
        "filename": file,
        "image size": image.size,
        "image height": image.height,
        "image width": image.width,
        "image format": image.format,
        "image mode": image.mode,
        "image is animated": getattr(image, "is_animated", False),
        "frames in image": getattr(image, "n_frames", 1)
    }

    exifdata = image.getexif()
    for tag_id in exifdata:    
        tag = TAGS.get(tag_id, tag_id).lower()
        data = exifdata.get(tag_id)
        info_dict[tag] = data
    return info_dict
        
def audio_video_process(dir_path, file):
    afile = os.path.join(dir_path, file)
    audio_video_dict = {}    
    audio_video_dict['filename'] = file   
    return audio_video_dict | ffmpeg.probe(afile)["streams"][0]

def txt_csv_process(dir_path, file):  # CSV 는 dataprep 의 메타 활용
    afile = os.path.join(dir_path, file)
    file_stat = os.stat(afile)
    c_time = os.path.getctime(afile)
    dt_c = datetime.datetime.fromtimestamp(c_time).strftime("%Y-%m-%d %H:%M:%S")
    txt_csv_dict = {'fileName': file, 'size(MB)':round(file_stat.st_size/1048576,1), 'created Date': dt_c}
    return txt_csv_dict
    

In [3]:
# f = magic.Magic(mime=True)
# f.from_file(r"C:\Users\jh\0py_dev\digitalship\sbs_collect\22000440975")

In [4]:
# kind = filetype.guess(os.path.join(dir_path,files[0]))
# print('File MIME type: %s' % kind.mime)

In [5]:
files = [ 'mmcv.txt','프로필_SAMPLE.pptx', 'Sample_abc.jpg', 'file_example_MP4_480_1_5MG.csv',
         '13.01', 'Sample_abc.txt', 'jb_04.csv', '126.html','file_example_MP4_480_1_5MG.mp4', 'video (2160p).mp4', 'P451172214_22000414182.txt','22000440975.mp3']
# files = ['22000440975.mp4', '22000440975.mp3', '22000440975_1.txt', '22000440975.csv', '22000440975']
# dir_path = r'C:\Users\jh\0py_dev\digitalship\sbs_collect' 
dir_path = 'D:\download'
file_dict = {}

try:
#     for file in files[:1]:
    for file in files:
        ftype = check_file_type(dir_path, file)            
        if ftype == 'image':
            file_dict[file] = image_process(dir_path,file)
        elif (ftype=='audio')|(ftype=='video'):
            file_dict[file] = audio_video_process(dir_path,file)
        elif (ftype=='txt')|(ftype=='csv'):
            file_dict[file] = txt_csv_process(dir_path,file)
        else:
            ftype = 'nothing'
        print(ftype)        
except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()    
    print(exc_type, exc_tb.tb_lineno, e)

txt
nothing
image
video
audio
image
csv
nothing
video
video
txt
audio


In [6]:
k = list(file_dict.keys())
for i in range(len(k)):
    print(i, file_dict[k[i]])

0 {'fileName': 'mmcv.txt', 'size(MB)': 0.0, 'created Date': '2023-06-05 16:02:17'}
1 {'filename': 'Sample_abc.jpg', 'image size': (420, 315), 'image height': 315, 'image width': 420, 'image format': 'JPEG', 'image mode': 'RGB', 'image is animated': False, 'frames in image': 1, 'resolutionunit': 2, 'exifoffset': 164, 'software': 'Adobe Photoshop CS3 Macintosh', 'orientation': 1, 'datetime': '2008:02:08 15:39:41', 'xresolution': 72.0, 'yresolution': 72.0}
2 {'filename': 'file_example_MP4_480_1_5MG.csv', 'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'Constrained Baseline', 'codec_type': 'video', 'codec_time_base': '1/60', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 480, 'height': 270, 'coded_width': 480, 'coded_height': 272, 'has_b_frames': 0, 'sample_aspect_ratio': '1:1', 'display_aspect_ratio': '16:9', 'pix_fmt': 'yuv420p', 'level': 30, 'chroma_location': 'left', 'refs': 1, 'is_avc': 'true', 'nal_length_s

In [7]:
file_dict[k[-1]]

{'filename': '22000440975.mp3',
 'index': 0,
 'codec_name': 'mp3',
 'codec_long_name': 'MP3 (MPEG audio layer 3)',
 'codec_type': 'audio',
 'codec_time_base': '1/44100',
 'codec_tag_string': '[0][0][0][0]',
 'codec_tag': '0x0000',
 'sample_fmt': 'fltp',
 'sample_rate': '44100',
 'channels': 2,
 'channel_layout': 'stereo',
 'bits_per_sample': 0,
 'r_frame_rate': '0/0',
 'avg_frame_rate': '0/0',
 'time_base': '1/14112000',
 'start_pts': 353600,
 'start_time': '0.025057',
 'duration_ts': 50671779840,
 'duration': '3590.687347',
 'bit_rate': '128000',
 'disposition': {'default': 0,
  'dub': 0,
  'original': 0,
  'comment': 0,
  'lyrics': 0,
  'karaoke': 0,
  'forced': 0,
  'hearing_impaired': 0,
  'visual_impaired': 0,
  'clean_effects': 0,
  'attached_pic': 0,
  'timed_thumbnails': 0},
 'tags': {'encoder': 'Lavc58.54'}}

In [111]:
import pandas as pd

In [125]:
pd.DataFrame(file_dict[k[0]], index=[0])

,fileName,size(MB),created Date
0,mmcv.txt,0.0,2023-06-05 16:02:17


In [127]:
pd.DataFrame(file_dict[k[1]])

,filename,image size,image height,image width,image format,image mode,image is animated,frames in image,resolutionunit,exifoffset,software,orientation,datetime,xresolution,yresolution
0,Sample_abc.jpg,420,315,420,JPEG,RGB,False,1,2,164,Adobe Photoshop CS3 Macintosh,1,2008:02:08 15:39:41,72.0,72.0
1,Sample_abc.jpg,315,315,420,JPEG,RGB,False,1,2,164,Adobe Photoshop CS3 Macintosh,1,2008:02:08 15:39:41,72.0,72.0


In [128]:
pd.DataFrame(file_dict[k[2]])

,filename,index,codec_name,codec_long_name,profile,codec_type,codec_time_base,codec_tag_string,codec_tag,width,...,time_base,start_pts,start_time,duration_ts,duration,bit_rate,bits_per_raw_sample,nb_frames,disposition,tags
default,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,1.0,NaN
dub,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
original,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
comment,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
lyrics,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
karaoke,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
forced,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
hearing_impaired,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
visual_impaired,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
clean_effects,file_example_MP4_480_1_5MG.csv,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN


In [129]:
pd.DataFrame(file_dict[k[3]])

,filename,index,codec_name,codec_long_name,codec_type,codec_time_base,codec_tag_string,codec_tag,sample_fmt,sample_rate,...,bits_per_sample,r_frame_rate,avg_frame_rate,time_base,start_pts,start_time,duration_ts,duration,bit_rate,disposition
attached_pic,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
clean_effects,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
comment,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
default,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
dub,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
forced,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
hearing_impaired,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
karaoke,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
lyrics,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0
original,13.01,0,mp3,MP3 (MPEG audio layer 3),audio,1/44100,[0][0][0][0],0x0000,fltp,44100,...,0,0/0,0/0,1/14112000,0,0.000000,789258053,55.928150,160000,0


In [130]:
pd.DataFrame(file_dict[k[4]])

,filename,image size,image height,image width,image format,image mode,image is animated,frames in image,resolutionunit,exifoffset,software,orientation,datetime,xresolution,yresolution
0,Sample_abc.txt,420,315,420,JPEG,RGB,False,1,2,164,Adobe Photoshop CS3 Macintosh,1,2008:02:08 15:39:41,72.0,72.0
1,Sample_abc.txt,315,315,420,JPEG,RGB,False,1,2,164,Adobe Photoshop CS3 Macintosh,1,2008:02:08 15:39:41,72.0,72.0


In [133]:
pd.DataFrame(file_dict[k[5]], index=[0])

,fileName,size(MB),created Date
0,jb_04.csv,0.0,2023-07-09 10:02:00


In [134]:
pd.DataFrame(file_dict[k[6]]) #, index=[0])

,filename,index,codec_name,codec_long_name,profile,codec_type,codec_time_base,codec_tag_string,codec_tag,width,...,time_base,start_pts,start_time,duration_ts,duration,bit_rate,bits_per_raw_sample,nb_frames,disposition,tags
default,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,1.0,NaN
dub,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
original,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
comment,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
lyrics,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
karaoke,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
forced,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
hearing_impaired,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
visual_impaired,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
clean_effects,file_example_MP4_480_1_5MG.mp4,0,h264,H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10,Constrained Baseline,video,1/60,avc1,0x31637661,480,...,1/30,0,0.000000,901,30.033333,301201,8,901,0.0,NaN
